In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import re
tf.random.set_seed(99)
BATCH_SIZE = 128
epochs = 10
valid = 0.2
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [2]:
# 데이터 읽기
# -- quoting : 특수 문자가 포함된 필드를 감쌀 때 처리하는 방법, 문자를 따옴표로 묶는 방법
import csv
df = pd.read_csv('data/labeledTrainData.tsv', header=0, delimiter='\t',quoting=3)


df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
train_data = pd.read_csv('data/labeledTrainData.tsv',header =0, delimiter = '\t', quoting =3)

def preprocessing(review, remove_stopwords = False):
    review_text = BeautifulSoup(review,'html.parser').get_text()
    
    # 특수문자 제거 # 영문자,숫자를 제외한 문자를 모드 변환 띄어쓰기로
    review_text = re.sub("\W"," ",review_text)    
    
    words = review_text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        clean_review = ' '.join(words)
        
    else :
        clean_review = ' '.join(words)
    
    return clean_review

clean_train_reviews = []

for review in train_data['review']:
    clean_train_reviews.append(preprocessing(review, remove_stopwords=True))

clean_train_reviews[0]


'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts 20 minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate wor

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)
print(text_sequences[0])

[410, 71, 425, 8957, 512, 2483, 116, 54, 882, 522, 179, 18946, 179, 11409, 167, 79, 14, 669, 2484, 118, 93, 10, 505, 4131, 167, 22, 213, 589, 2361, 1205, 11409, 72, 4896, 72, 641, 2, 257, 71, 11, 306, 1683, 492, 1157, 3309, 8957, 417, 800, 3387, 17, 447, 607, 1516, 15, 4528, 1875, 1010, 148, 348, 1455, 750, 2452, 4, 8957, 424, 71, 643, 70, 241, 95, 547, 8957, 26374, 26375, 121, 1, 8957, 327, 8, 47, 20, 327, 169, 10, 210, 638, 641, 2, 117, 295, 388, 733, 124, 15761, 3361, 1517, 582, 741, 10167, 934, 11746, 829, 1251, 1423, 366, 8957, 225, 15, 584, 8957, 22505, 2300, 13630, 741, 10167, 27, 28950, 346, 16, 41, 18947, 1516, 394, 11410, 167, 4018, 8957, 116, 633, 505, 80, 4, 8957, 1444, 386, 2193, 115, 1943, 2529, 582, 17, 60, 101, 4947, 5239, 264, 1280, 26376, 15, 582, 498, 751, 643, 637, 3, 400, 166, 452, 115, 622, 3310, 1172, 690, 48, 1190, 228, 1, 16, 4, 8957, 3, 513, 62, 25, 16, 646, 135, 235, 96, 7553, 607, 3486, 8957, 37737, 1888, 1, 130, 348, 1455, 251, 3, 874, 16, 42, 1502, 1009, 2

In [5]:
word_vocab = tokenizer.word_index
word_vocab["<PAD>"]=0 
# print(word_vocab)

In [6]:
data_configs = {}
data_configs['vocab']= word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
data_configs['vocab_size']

75684

In [7]:
MAX_SEQUENCE_LENGTH = 174

train_inputs =  pad_sequences(text_sequences,maxlen = MAX_SEQUENCE_LENGTH, padding = 'post')
print(train_inputs.shape)
train_labels = np.array(train_data['sentiment'])
print(train_labels.shape)

(25000, 174)
(25000,)


In [8]:
kargs = {
    'vocab_size' : data_configs['vocab_size'],
    'embedding_size' : 128,
    'num_filters' : 100,
    'dropout_rate' : 0.5,
    'hidden_dimension' : 250,
    'output_dimension' : 1
    
}

In [9]:
class CNN(tf.keras.Model):
    
    def __init__(self,**kargs):
        super(CNN,self).__init__()
        self.embedding = tf.keras.layers.Embedding(input_dim = kargs['vocab_size'],
                                            output_dim = kargs['embedding_size'])
        
        self.conv_list = [tf.keras.layers.Conv1D(filters = kargs['num_filters'],
                                       kernel_size = kernel_size,
                                       padding ='valid',
                                    activation = tf.keras.activations.relu,
                                        kernel_constraint = tf.keras.constraints.MaxNorm(max_value=3.)                                       
                                       )
                             for kernel_size in [3,4,5]]
        self.pooling = tf.keras.layers.GlobalMaxPool1D()
        self.dropout = tf.keras.layers.Dropout(kargs['dropout_rate'])
        self.fc1 = tf.keras.layers.Dense(units = kargs['hidden_dimension'],
                                        activation = tf.keras.activations.relu,
                                         # 가중치가 너무 커지는것을 방지
                                         kernel_constraint = tf.keras.constraints.MaxNorm(max_value = 3.)                                        
                                        )
        self.fc2 = tf.keras.layers.Dense(units=kargs['output_dimension'],
                                        activation = tf.keras.activations.sigmoid,
                                         kernel_constraint = tf.keras.constraints.MaxNorm(max_value=3.)
                                        )
    def call(self,x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list] , axis =1)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [10]:
model = CNN(**kargs)
model.compile(optimizer = tf.keras.optimizers.Adam(1e-4),
             loss = tf.keras.losses.BinaryCrossentropy(),
              metrics= [tf.keras.metrics.BinaryAccuracy(name = "accuracy")]
             )

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# earlystopping 
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta=0.0001, patience=2)
model_name = 'rnn_classifier'
checkpoint_path = './data_out/{}/weight2.h5'.format(model_name)
checkpoint_dir =  os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
    print("{} - exists".format(checkpoint_dir))
else :
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} - complete".format(checkpoint_dir))

cp_callback= ModelCheckpoint(
    checkpoint_path, monitor ='val_accuracy', verbose =1, save_best_only =True, save_weights_only = True
)

./data_out/rnn_classifier - exists


In [ ]:
history = model.fit(train_inputs, train_labels, batch_size = 512, epochs =  10,
                    validation_split = 0.2 , callbacks = [earlystop, cp_callback])

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
